In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [18]:
pd.read_csv(path/'teams.csv')

,team,team_code,team_1617,team_1718,team_1819,team_1920,team_2021
0,Arsenal,3,1.0,1.0,1.0,1.0,1.0
1,Bournemouth,91,2.0,2.0,2.0,3.0,NaN
2,Burnley,90,3.0,4.0,4.0,5.0,4.0
3,Chelsea,8,4.0,5.0,6.0,6.0,5.0
4,Crystal Palace,31,5.0,6.0,7.0,7.0,6.0
5,Everton,11,6.0,7.0,8.0,8.0,7.0
6,Hull City,88,7.0,NaN,NaN,NaN,NaN
7,Leicester City,13,8.0,9.0,11.0,9.0,9.0
8,Liverpool,14,9.0,10.0,12.0,10.0,11.0
9,Manchester City,43,10.0,11.0,13.0,11.0,12.0


In [37]:
# read in latest training set
df_train = pd.read_csv(path/'train_v4.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2020-21']

# name for current season
season_names = ['2021']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2021', 11, range(13,33))

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
90387,Luke_Freeman,38,3,0,Sheffield United,Southampton,0.348678,0.526987,False,0,...,1,3,0.0,-30,8,38,0,2020-07-26T15:00:00Z,1920,0.25
90388,Kelland_Watts,38,2,16,Newcastle United,Liverpool,0.611449,2.406333,True,1,...,3,1,0.0,3319,4336,1017,0,2020-07-26T15:00:00Z,1920,1.00
90389,Shkodran_Mustafi,38,2,0,Arsenal,Watford,1.452343,0.472917,True,0,...,2,3,0.0,-1929,135,2064,0,2020-07-26T15:00:00Z,1920,0.00
90390,Mace_Goodridge,38,3,0,Burnley,Brighton and Hove Albion,0.370648,0.541184,True,0,...,2,1,0.0,1,26,25,0,2020-07-26T15:00:00Z,1920,1.00
90391,Dan_Gosling,38,3,90,Bournemouth,Everton,0.565681,1.005607,False,2,...,3,1,1.0,-87,95,182,0,2020-07-26T15:00:00Z,1920,1.00
90392,Tammy_Abraham,38,4,12,Chelsea,Wolverhampton Wanderers,1.999543,0.811175,True,1,...,0,2,11.0,-10050,10886,20936,0,2020-07-26T15:00:00Z,1920,1.00
90393,Nicolás_Otamendi,38,2,0,Manchester City,Norwich,2.430397,0.327574,True,0,...,0,5,0.0,-1574,7965,9539,0,2020-07-26T15:00:00Z,1920,1.00
90394,James_Daly,38,4,0,Crystal Palace,Tottenham Hotspur,0.430493,1.604904,True,0,...,1,1,0.0,125,165,40,0,2020-07-26T15:00:00Z,1920,0.00
90395,Declan_Rice,38,3,90,West Ham United,Aston Villa,0.709989,0.553818,True,5,...,1,1,3.0,15208,18372,3164,0,2020-07-26T15:00:00Z,1920,1.00
90396,Connor_Wickham,38,4,0,Crystal Palace,Tottenham Hotspur,0.430493,1.604904,True,0,...,1,1,0.0,1567,2425,858,0,2020-07-26T15:00:00Z,1920,0.00


In [6]:
# df_train[(df_train['gw']==24) & (df_train['season']=='1920') & (df_train['team']=='Liverpool')]

In [7]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

39

In [7]:
# last_gw = 28

In [8]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [9]:
df_latest.drop_duplicates(inplace=True)

In [10]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Chris_Mepham,38,2,0,Bournemouth,Everton,0.565681,1.005607,False,0,...,45168,3,1,0.0,-238,13,251,0,2020-07-26T15:00:00Z,1920
1,Paulo_Gazzaniga,38,1,0,Tottenham Hotspur,Crystal Palace,1.604904,0.430493,False,0,...,113721,1,1,0.0,-1374,1716,3090,0,2020-07-26T15:00:00Z,1920
2,John_Egan,38,2,90,Sheffield United,Southampton,0.348678,0.526987,False,1,...,518244,1,3,0.0,2202,31171,28969,0,2020-07-26T15:00:00Z,1920
3,Max_Meyer,38,3,0,Crystal Palace,Tottenham Hotspur,0.430493,1.604904,True,0,...,5985,1,1,0.0,-8,14,22,0,2020-07-26T15:00:00Z,1920
4,Fabian_Delph,38,2,0,Everton,Bournemouth,1.005607,0.565681,True,0,...,12733,3,1,0.0,-51,22,73,0,2020-07-26T15:00:00Z,1920
5,James_Justin,38,2,90,Leicester City,Manchester United,1.076834,1.753447,True,1,...,38893,2,0,0.0,-5516,908,6424,0,2020-07-26T15:00:00Z,1920
6,Che_Adams,38,4,84,Southampton,Sheffield United,0.526987,0.348678,True,13,...,31051,1,3,64.0,3348,4349,1001,0,2020-07-26T15:00:00Z,1920
7,Mahmoud Ahmed_Ibrahim Hassan,38,3,89,Aston Villa,West Ham United,0.553818,0.709989,False,2,...,153177,1,1,11.0,47220,50470,3250,0,2020-07-26T15:00:00Z,1920
8,Tyreece_John-Jules,38,4,0,Arsenal,Watford,1.452343,0.472917,True,0,...,521,2,3,0.0,33,138,105,0,2020-07-26T15:00:00Z,1920
9,Adrian_Mariappa,38,2,45,Watford,Arsenal,0.472917,1.452343,False,1,...,18383,2,3,0.0,-49,377,426,0,2020-07-26T15:00:00Z,1920


In [11]:
df_latest.shape

(666, 34)

In [12]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [13]:
last_play_proba.head()

,player,play_proba
0,Shkodran_Mustafi,0.0
1,Héctor_Bellerín,1.0
2,Sead_Kolasinac,1.0
3,Ainsley_Maitland-Niles,1.0
4,Sokratis_Papastathopoulos,1.0


In [14]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [15]:
df_latest.drop_duplicates(inplace=True)

In [16]:
df_train.drop_duplicates(inplace=True)

In [17]:
df_train.shape

(89771, 35)

In [18]:
df_latest.shape

(666, 35)

In [19]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [20]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
90432,Tommy_Doyle,38,3,0,Manchester City,Norwich,2.430397,0.327574,True,0,...,0,5,0.0,-2,22,24,0,2020-07-26T15:00:00Z,1920,1.0
90433,Joseph_Anang,38,1,0,West Ham United,Aston Villa,0.709989,0.553818,True,0,...,1,1,0.0,70,270,200,0,2020-07-26T15:00:00Z,1920,1.0
90434,Erik_Pieters,38,2,90,Burnley,Brighton and Hove Albion,0.370648,0.541184,True,3,...,2,1,2.0,139816,144388,4572,1,2020-07-26T15:00:00Z,1920,1.0
90435,Japhet_Tanganga,38,2,0,Tottenham Hotspur,Crystal Palace,1.604904,0.430493,False,0,...,1,1,0.0,7999,14840,6841,0,2020-07-26T15:00:00Z,1920,1.0
90436,Ravel_Morrison,38,3,0,Sheffield United,Southampton,0.348678,0.526987,False,0,...,1,3,0.0,-23,7,30,0,2020-07-26T15:00:00Z,1920,0.0


In [21]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v3.csv')

In [3]:
df_train_new = pd.read_csv(path/'train_v4.csv', index_col=0, dtype={'season':str})

In [23]:
# df_train_new[(df_train_new['team'] == 'Manchester United') & 
#              (df_train_new['gw'] == 32) & 
#              (df_train_new['season'] == '1920')]

In [4]:
df_train_new

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season
0,Aaron_Cresswell,1,2,0,West Ham United,Chelsea,NaN,NaN,False,0,...,0.0,0,0,0,0,2016-08-15T19:00:00Z,1617,NaN,0.895471,2.243698
1,Aaron_Lennon,1,3,15,Everton,Tottenham Hotspur,NaN,NaN,True,1,...,0.0,0,0,0,0,2016-08-13T14:00:00Z,1617,NaN,1.057509,1.433690
2,Aaron_Ramsey,1,3,60,Arsenal,Liverpool,NaN,NaN,True,2,...,23.0,0,0,0,0,2016-08-14T15:00:00Z,1617,NaN,1.944129,1.465860
3,Abdoulaye_Doucouré,1,3,0,Watford,Southampton,NaN,NaN,False,0,...,0.0,0,0,0,0,2016-08-13T14:00:00Z,1617,NaN,0.704200,0.796805
4,Abdul Rahman_Baba,1,2,0,Chelsea,West Ham United,NaN,NaN,True,0,...,0.0,0,0,0,0,2016-08-15T19:00:00Z,1617,NaN,2.243698,0.895471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90432,Tommy_Doyle,38,3,0,Manchester City,Norwich,2.430397,0.327574,True,0,...,0.0,-2,22,24,0,2020-07-26T15:00:00Z,1920,1.0,2.727025,0.198300
90433,Joseph_Anang,38,1,0,West Ham United,Aston Villa,0.709989,0.553818,True,0,...,0.0,70,270,200,0,2020-07-26T15:00:00Z,1920,1.0,0.739196,0.338194
90434,Erik_Pieters,38,2,90,Burnley,Brighton and Hove Albion,0.370648,0.541184,True,3,...,2.0,139816,144388,4572,1,2020-07-26T15:00:00Z,1920,1.0,0.441799,0.476156
90435,Japhet_Tanganga,38,2,0,Tottenham Hotspur,Crystal Palace,1.604904,0.430493,False,0,...,0.0,7999,14840,6841,0,2020-07-26T15:00:00Z,1920,1.0,2.113981,0.495374


In [5]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = 1
# current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [6]:
fixtures.head(20)

,home_team,away_team,gw
0,Fulham,Arsenal,1
1,Crystal Palace,Southampton,1
2,West Ham United,Newcastle United,1
3,Liverpool,Leeds,1
4,West Bromwich Albion,Leicester City,1
5,Tottenham Hotspur,Everton,1
6,Brighton and Hove Albion,Chelsea,1
7,Sheffield United,Wolverhampton Wanderers,1
8,Burnley,Manchester United,28
9,Manchester City,Aston Villa,28


In [10]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [13]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,1,Fulham,Arsenal,54,3
1,1,Crystal Palace,Southampton,31,20
2,1,West Ham United,Newcastle United,21,4
3,1,Liverpool,Leeds,14,2
4,1,West Bromwich Albion,Leicester City,35,13


In [26]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [30]:
all_players[all_players['play_proba_2021'] == '50']

,full_name,play_proba_2021,position_2021,id_2021,cost_2021,team_2021
122,Christian_Pulisic,50,3,119,85,8
220,James_Maddison,50,3,231,70,13
256,Konstantinos_Tsimikas,50,2,242,50,14
262,Alex_Oxlade-Chamberlain,50,3,248,65,14
337,Matt_Ritchie,50,3,322,50,4


In [50]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2021', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_2021', 'cost_2021', 'play_proba_2021']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2021':'position',
                        'cost_2021':'price',
                        'play_proba_2021':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [52]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2021', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_2021', 'cost_2021', 'play_proba_2021']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2021':'position',
                        'cost_2021':'price',
                        'play_proba_2021':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [47]:
all_players[all_players['full_name'] == 'Fabricio_Agosto Ramírez']

,full_name,play_proba_2021,position_2021,id_2021,cost_2021,team_2021
180,Fabricio_Agosto Ramírez,None,1,171,40,54


In [53]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,1,Arsenal,Fulham,Mesut_Özil,3,70,None,False
1,1,Arsenal,Fulham,Sokratis_Papastathopoulos,2,50,None,False
2,1,Arsenal,Fulham,David_Luiz Moreira Marinho,2,55,None,False
3,1,Arsenal,Fulham,Pierre-Emerick_Aubameyang,3,120,None,False
4,1,Arsenal,Fulham,Cédric_Soares,2,50,None,False


In [54]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,1,Fulham,Arsenal,Fabricio_Agosto Ramírez,1,40,None,True
1,1,Fulham,Arsenal,Kevin_McDonald,3,50,None,True
2,1,Fulham,Arsenal,Maxime_Le Marchand,2,45,None,True
3,1,Fulham,Arsenal,Stefan_Johansen,3,50,None,True
4,1,Fulham,Arsenal,Tom_Cairney,3,55,None,True


In [36]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

recent_mins = df_train_new[(df_train_new['gw'] >= 39 - 5) & 
                           (df_train_new['season'] == '1920')][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 33.2,
 'Aaron_Cresswell': 86.0,
 'Aaron_Mooy': 42.6,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 76.0,
 'Abdoulaye_Doucouré': 82.4,
 'Adalberto_Peñaranda': 0.0,
 'Adam_Idah': 12.6,
 'Adam_Lallana': 0.0,
 'Adam_Masina': 63.6,
 'Adam_Smith': 62.0,
 'Adam_Webster': 90.0,
 'Adama_Traoré': 62.6,
 'Adrian_Mariappa': 36.0,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 51.0,
 'Ainsley_Maitland-Niles': 21.0,
 'Akin_Famewo': 1.0,
 'Albian_Ajeti': 0.8,
 'Alex_Cochrane': 0.0,
 'Alex_Iwobi': 24.0,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 40.6,
 'Alexander_Tettey': 86.0,
 'Alexandre Nascimento_Costa Silva': 0.0,
 'Alexandre_Jankewitz': 0.0,
 'Alexandre_Lacazette': 72.6,
 'Alexis_Mac Allister': 37.8,
 'Alfie_Lewis': 0.0,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 5.0,
 'Alisson_Ramses Becker': 90.0,
 'Allan_Saint-Maximin': 45.0,
 'Andre_Gray': 4.8,
 'Andreas_Christensen': 27.0,
 'Andreas_Pereira': 0.0,
 'Andrew_Robertson': 81.0,
 'Andrew_Surman': 

In [58]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2021'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2021'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last five fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

remaining_season_df['minutes'] = 90

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [59]:
remaining_season_df

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,1,Fulham,Arsenal,Fabricio_Agosto Ramírez,1,4.0,1.0,True,0.318038,1.535042,1920,90.0
1,1,Fulham,Arsenal,Kevin_McDonald,3,5.0,1.0,True,0.318038,1.535042,1920,90.0
2,1,Fulham,Arsenal,Maxime_Le Marchand,2,4.5,1.0,True,0.318038,1.535042,1920,90.0
3,1,Fulham,Arsenal,Stefan_Johansen,3,5.0,1.0,True,0.318038,1.535042,1920,90.0
4,1,Fulham,Arsenal,Tom_Cairney,3,5.5,1.0,True,0.318038,1.535042,1920,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19185,38,Manchester United,Wolverhampton Wanderers,Tahith_Chong,3,4.5,0.0,False,1.890316,0.785020,1920,0.0
19186,38,Manchester United,Wolverhampton Wanderers,James_Garner,3,4.5,1.0,False,1.890316,0.785020,1920,90.0
19187,38,Manchester United,Wolverhampton Wanderers,Brandon_Williams,2,4.5,1.0,False,1.890316,0.785020,1920,90.0
19188,38,Manchester United,Wolverhampton Wanderers,Teden_Mengi,2,4.0,1.0,False,1.890316,0.785020,1920,90.0


In [60]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,1,Fulham,Arsenal,Fabricio_Agosto Ramírez,1,4.0,1.00,True,0.318038,1.535042,1920,90.0
1,1,Fulham,Arsenal,Kevin_McDonald,3,5.0,1.00,True,0.318038,1.535042,1920,90.0
2,1,Fulham,Arsenal,Maxime_Le Marchand,2,4.5,1.00,True,0.318038,1.535042,1920,90.0
3,1,Fulham,Arsenal,Stefan_Johansen,3,5.0,1.00,True,0.318038,1.535042,1920,90.0
4,1,Fulham,Arsenal,Tom_Cairney,3,5.5,1.00,True,0.318038,1.535042,1920,90.0
5,1,Fulham,Arsenal,Tim_Ream,2,4.5,1.00,True,0.318038,1.535042,1920,90.0
6,1,Fulham,Arsenal,Anthony_Knockaert,3,5.5,1.00,True,0.318038,1.535042,1920,90.0
7,1,Fulham,Arsenal,Michael_Hector,2,4.5,1.00,True,0.318038,1.535042,1920,90.0
8,1,Fulham,Arsenal,Cyrus_Christie,2,4.5,1.00,True,0.318038,1.535042,1920,90.0
9,1,Fulham,Arsenal,Neeskens_Kebano,3,5.0,1.00,True,0.318038,1.535042,1920,90.0


In [61]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')